## Version Check

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.0.11'

## Connecting

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

## Declare a Mapping

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s'>" % (self.name, self.fullname, self.password)

## Create a Schema

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

## Create an Instance of the Mapped Class

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user.name

'ed'

In [8]:
ed_user.password

'edspassword'

In [9]:
str(ed_user.id)

'None'

## Creating a Session

In [12]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [13]:
session = Session()

## Adding and Updating Objects

In [17]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

In [18]:
our_user = session.query(User).filter_by(name='ed').first()
our_user

2016-01-29 22:03:47,676 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2016-01-29 22:03:47,677 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones', 'edspassword')


2016-01-29 22:03:47,678 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2016-01-29 22:03:47,679 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', password='edspassword'>

In [20]:
ed_user is our_user

True

In [21]:
session.add_all([
        User(name='wendy', fullname='Wendy Williams', password='foobar'),
        User(name='mary', fullname='Mary Contrary', password='xxg527'),
        User(name='fred', fullname='Fred Flinstone', password='blah')
    ])

In [22]:
ed_user.password = 'f8s7ccs'

In [23]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs'>])

In [24]:
session.new

IdentitySet([<User(name='fred', fullname='Fred Flinstone', password='blah'>, <User(name='wendy', fullname='Wendy Williams', password='foobar'>, <User(name='mary', fullname='Mary Contrary', password='xxg527'>])

In [25]:
session.commit()

2016-01-29 22:06:24,332 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET password=? WHERE users.id = ?


2016-01-29 22:06:24,333 INFO sqlalchemy.engine.base.Engine ('f8s7ccs', 2)


INFO:sqlalchemy.engine.base.Engine:('f8s7ccs', 2)


2016-01-29 22:06:24,333 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2016-01-29 22:06:24,336 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')


INFO:sqlalchemy.engine.base.Engine:('fred', 'Fred Flinstone', 'blah')


2016-01-29 22:06:24,337 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [26]:
ed_user.id

2016-01-29 22:06:33,993 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2016-01-29 22:06:33,994 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2016-01-29 22:06:33,995 INFO sqlalchemy.engine.base.Engine (2,)


INFO:sqlalchemy.engine.base.Engine:(2,)


2

## Rolling Back

In [27]:
ed_user.name = 'Edwardo'

In [28]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [29]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2016-01-29 22:10:00,270 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE users SET name=? WHERE users.id = ?


2016-01-29 22:10:00,271 INFO sqlalchemy.engine.base.Engine ('Edwardo', 2)


INFO:sqlalchemy.engine.base.Engine:('Edwardo', 2)


2016-01-29 22:10:00,272 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


...

2016-01-29 22:10:00,275 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2016-01-29 22:10:00,275 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs'>,
 <User(name='fakeuser', fullname='Invalid', password='12345'>]

In [30]:
session.rollback()
ed_user.name

2016-01-29 22:10:21,795 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


2016-01-29 22:10:21,797 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2016-01-29 22:10:21,798 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?


2016-01-29 22:10:21,798 INFO sqlalchemy.engine.base.Engine (2,)


INFO:sqlalchemy.engine.base.Engine:(2,)


'ed'

In [31]:
fake_user in session

False

In [32]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2016-01-29 22:13:13,543 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)


2016-01-29 22:13:13,544 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


INFO:sqlalchemy.engine.base.Engine:('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='f8s7ccs'>]

## Querying

In [33]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2016-01-29 22:17:33,725 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id


2016-01-29 22:17:33,726 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ad Ed Jones
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [34]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2016-01-29 22:19:38,585 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users


2016-01-29 22:19:38,586 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ad Ed Jones
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [35]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2016-01-29 22:32:13,490 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users


2016-01-29 22:32:13,491 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ad', fullname='Ed Jones', password='edspassword'> ad
<User(name='ed', fullname='Ed Jones', password='f8s7ccs'> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar'> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527'> mary
<User(name='fred', fullname='Fred Flinstone', password='blah'> fred


In [36]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2016-01-29 22:33:11,932 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS name_label 
FROM users


2016-01-29 22:33:11,933 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


ad
ed
wendy
mary
fred


In [37]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')

for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2016-01-29 22:35:07,989 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


INFO:sqlalchemy.engine.base.Engine:SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias


2016-01-29 22:35:07,990 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


<User(name='ad', fullname='Ed Jones', password='edspassword'>
<User(name='ed', fullname='Ed Jones', password='f8s7ccs'>
<User(name='wendy', fullname='Wendy Williams', password='foobar'>
<User(name='mary', fullname='Mary Contrary', password='xxg527'>
<User(name='fred', fullname='Fred Flinstone', password='blah'>


In [38]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2016-01-29 22:36:09,934 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?


2016-01-29 22:36:09,935 INFO sqlalchemy.engine.base.Engine (2, 1)


INFO:sqlalchemy.engine.base.Engine:(2, 1)


<User(name='ed', fullname='Ed Jones', password='f8s7ccs'>
<User(name='wendy', fullname='Wendy Williams', password='foobar'>


In [39]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2016-01-29 22:36:57,359 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2016-01-29 22:36:57,360 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ad
ed


In [40]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2016-01-29 22:37:55,740 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?


2016-01-29 22:37:55,740 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)


INFO:sqlalchemy.engine.base.Engine:('Ed Jones',)


ad
ed


In [41]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2016-01-29 22:38:45,614 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?


2016-01-29 22:38:45,614 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')


INFO:sqlalchemy.engine.base.Engine:('ed', 'Ed Jones')


<User(name='ed', fullname='Ed Jones', password='f8s7ccs'>
